# TPC4:  Ficheiros CSV com listas e funções de agregação

Cria um programa em Python que implementa um conversor de um ficheiro CSV (Comma separated values) para o formato JSON.
Para se poder realizar a conversão pretendida, é importante saber que a primeira linha do CSV dado funciona como cabeçalho
que define o que representa cada coluna.
Por exemplo, o seguinte ficheiro "`alunos.csv`":

```
Número,Nome,Curso
3162,Cândido Faísca,Teatro
7777,Cristiano Ronaldo,Desporto
264,Marcelo Sousa,Ciência Política
```
Que corresponde a uma tabela com 3 registos de informação: a primeira linha de cabeçalho identifica os campos de cada registo, `Número`, `Nome`, `Curso`, e as linhas seguintes contêm os registos de informação.

No entanto, os CSV recebidos poderão conter algumas extensões cuja semântica se explica a seguir:

### 1. Listas

No cabeçalho, cada campo poderá ter um número `N` que representará o número de colunas que esse campo abrange.
Por exemplo, imaginemos que ao exemplo anterior se acrescentou um campo `Notas`, com `N = 5` ("`alunos2.csv`"):

```
Número,Nome,Curso,Notas{5},,,,,
3162,Cândido Faísca,Teatro,12,13,14,15,16
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,18
```

Isto significa que o campo Notas abrange 5 colunas (reparem que se colocaram os campos que sobram a vazio, para o
**CSV bater certo**).

### 2. Listas com um intervalo de tamanhos

Para além de um tamanho único, podemos também definir um intervalo de tamanhos `{N, M}`, significando que o número de
colunas de um certo campo pode ir de `N` até `M` ("`alunos3.csv`"):

```
Número,Nome,Curso,Notas{3,5},,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,
```

À semelhança do ponto anterior, havendo colunas vazias, os separadores têm de estar lá, o número de colunas deverá ser sempre igual ao valor máximo de colunas, poderão é estar preenchidas com informação ou não.

### 3. Funções de agregação

Para além de listas, podemos ter funções de agregação, aplicadas a essas listas.
Veja os seguintes exemplos: "`alunos4.csv`"

```
Número,Nome,Curso,Notas{3,5}::sum,,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,
```

 e "`alunos5.csv`":

 ```
Número,Nome,Curso,Notas{3,5}::media,,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,
 ```

## Resultados esperados

O resultado final esperado é um ficheiro JSON resultante da conversão dum ficheiro CSV.
Por exemplo, o ficheiro "`alunos.csv`" (original), deveria ser transformado no seguinte ficheiro "`alunos.json`":

```
[
  {
    "Número": "3612",
    "Nome": "Cândido Faísca",
    "Curso": "Teatro"
  },
  {
    "Número": "7777",
    "Nome": "Cristiano Ronaldo",
    "Curso": "Desporto"
  },
  {
    "Número": "264",
    "Nome": "Marcelo Sousa",
    "Curso": "Ciência Política"
  }
]
```

No caso de existirem listas, os campos que representam essas listas devem ser mapeados para listas em JSON ("`alunos2.csv ==> alunos2.json`"):

```
[
  {
    "Número": "3612",
    "Nome": "Cândido Faísca",
    "Curso": "Teatro",
    "Notas": [12,13,14,15,16]
  },
  {
    "Número": "7777",
    "Nome": "Cristiano Ronaldo",
    "Curso": "Desporto",
    "Notas": [17,12,20,11,12]
  },
  {
    "Número": "264",
    "Nome": "Marcelo Sousa",
    "Curso": "Ciência Política",
    "Notas": [18,19,19,20,18]
  }
]
```

Nos casos em que temos uma lista com uma função de agregação, o processador deve executar a função associada à lista, e
colocar o resultado no JSON, identificando na chave qual foi a função executada ("`alunos4.csv ==> alunos4.json`"):

```
[
  {
    "Número": "3612",
    "Nome": "Cândido Faísca",
    "Curso": "Teatro",
    "Notas_sum": 39
  },
  {
    "Número": "7777",
    "Nome": "Cristiano Ronaldo",
    "Curso": "Desporto",
    "Notas_sum": 72
  },
  {
    "Número": "264",
    "Nome": "Marcelo Sousa",
    "Curso": "Ciência Política",
    "Notas_sum": 76
  }
]
```

# Funções de agregação implementadas

<li><b>sum</b>: calcula a soma de todos os elementos presentes na lista;</li> <br>

<li><b>media</b>: calcula o valor médio de todos os elementos presentes na lista;</li> <br>

<li><b>max</b>: calcula o valor máximo presente na lista;</li> <br>

<li><b>min</b>: calcula o valor mínimo presente na lista;</li> <br>

<li><b>prod</b>: calcula o produto de todos os elementos presentes na lista;</li> <br>

<li><b>len/length</b>: calcula o comprimento da lista;</li> <br>

<li><b>reverse</b>: retorna a lista organizada em ordem inversa;</li> <br>

<li><b>sorted/sorted_asc</b>: retorna a lista ordenada por ordem ascendente;</li> <br>

<li><b>sorted_desc</b>: retorna a lista ordenada por ordem descendente;</li> <br>

# Programa implementado

In [1]:
import re
import json
import sys
from math import prod

def get_json_file_path(csv_file_path):
    fp_splited = re.split(r"\.", csv_file_path)
    json_file_path = ""
    
    if fp_splited is not None:
        for i in range(len(fp_splited)-1):
            json_file_path += fp_splited[i]
        json_file_path += ".json"
        
    return json_file_path


def is_numbers_list(lista):
    for n in lista:
        match = re.match(r"^[+-]?\d*\.?\d+$", str(n))
        if match is None:
            return False
               
    return True

def convert_str_list(lista):
    for i in range(len(lista)):
        lista[i] = float(lista[i])
        if lista[i].is_integer():
            lista[i] = int(lista[i])
    
    return lista

def manage_functions(funcao, lista):
    r = ""
    lista_nums = is_numbers_list(lista)
    
    if lista_nums:    
        if funcao == 'sum':
            r = sum(lista)
            if isinstance(r, float) and r.is_integer():
                r = int(r)
                
        elif funcao == 'media':
            r = sum(lista)/len(lista)
            if isinstance(r, float) and r.is_integer():
                r = int(r)
                
        elif funcao == 'max':
            r = max(lista)
            if isinstance(r, float) and r.is_integer():
                r = int(r)
                
        elif funcao == 'min':
            r = min(lista)
            if isinstance(r, float) and r.is_integer():
                r = int(r)
                
        elif funcao == 'prod':
            r = prod(lista)
            if isinstance(r, float) and r.is_integer():
                r = int(r)
                
        elif funcao == 'len' or funcao == 'length':
            r = len(lista)
            if isinstance(r, float) and r.is_integer():
                r = int(r)
                
        elif funcao == 'reverse':
            lista.reverse()
            r = lista
            
        elif funcao == 'sorted' or funcao == 'sorted_asc':
            r = sorted(lista)
            
        elif funcao == 'sorted_desc':
            r = sorted(lista, reverse=True)
            
    return r
    
def convert_csv_to_json(file_path):
    file_read = open(file_path, "r")
    i = 0
    lista_dict_json = []
    campos = []
    for line in file_read:
        val_linha = re.split(r",(?![^{]*\})", line)
        
        if val_linha[-1][-1] == '\n':
            val_linha[-1] = val_linha[-1][0:-1]
        
        if i == 0:  
            campos = val_linha      
        else:
            dict_json = dict()
            j = 0
            while j < len(campos):
                val = val_linha[j]
                
                if val is not None:
                    match1 = re.match(r"(?P<nome>\w+){(?P<N>\d+),(?P<M>\d+)}::(?P<func>[\w ]*[\w]+)", campos[j])
                    match2 = re.match(r"(?P<nome>\w+){(?P<N>\d+)}::(?P<func>[\w ]*[\w]+)", campos[j])
                    match3 = re.match(r"(?P<nome>\w+){(?P<N>\d+),(?P<M>\d+)}", campos[j])
                    match4 = re.match(r"(?P<nome>\w+){(?P<N>\d+)}", campos[j])
                    
                    if match1:
                        N = int(match1.group('N'))
                        M = int(match1.group('M'))                        
                        aux = []
                        for k in range(j, j+M):
                            if val_linha[k] != "":
                                aux.append(val_linha[k])
                        
                        if len(aux) >= N and len(aux) <= M:
                            flag_lista_nums = is_numbers_list(aux)
                            if flag_lista_nums:    
                                aux = convert_str_list(aux)
                            
                            key = match1.group('nome') + '_' + match1.group('func')
                            dict_json[key] = manage_functions(match1.group('func'), aux)
                        
                        j += M+1
                        
                    elif match2:
                        N = int(match2.group('N'))                      
                        aux = []
                        for k in range(j, j+N):
                            if val_linha[k] != "":
                                aux.append(val_linha[k])

                        if len(aux) == N:
                            flag_lista_nums = is_numbers_list(aux)
                            if flag_lista_nums:    
                                aux = convert_str_list(aux)
                            
                            key = match2.group('nome') + '_' + match2.group('func')
                            dict_json[key] = manage_functions(match2.group('func'), aux)
                            
                        j += N+1
                    
                    elif match3:
                        N = int(match3.group('N'))
                        M = int(match3.group('M'))                        
                        aux = []
                        for k in range(j, j+M):
                            if val_linha[k] != "":
                                aux.append(val_linha[k])
                        
                        if len(aux) >= N and len(aux) <= M:
                            flag_lista_nums = is_numbers_list(aux)
                            if flag_lista_nums:    
                                aux = convert_str_list(aux)
                            
                            key = match3.group('nome')
                            dict_json[key] = aux
                            
                        j += M+1
                    
                    elif match4:
                        N = int(match4.group('N'))                      
                        aux = []
                        for k in range(j, j+N):
                            if val_linha[k] != "":
                                aux.append(val_linha[k])
                        
                        if len(aux) == N:
                            flag_lista_nums = is_numbers_list(aux)
                            if flag_lista_nums:    
                                aux = convert_str_list(aux)
                            
                            key = match4.group('nome')
                            dict_json[key] = aux
                        
                        j += N+1
                    
                    else:
                        dict_json[campos[j]] = val      
                        j += 1
                        
            lista_dict_json.append(dict_json)                 
        i += 1
        
    file_read.close()
    
    file_write = open(get_json_file_path(file_path), "w")
    json.dump(lista_dict_json, file_write, ensure_ascii=False, indent=2)
    file_write.close()


def main():
    if len(sys.argv) >= 2:
        convert_csv_to_json(sys.argv[1])
    else:
        file_path = input("Introduza o caminho do ficheiro .csv a converter: ")
        convert_csv_to_json(file_path)

main()